In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os, sys
sys.path.append(os.path.join("../.."))

import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import privugger as pv

In [ ]:
def age():
    